In [1]:
# Importing libraries
import os, shutil, random
import tensorflow as tf 
from tensorflow.keras.utils import image_dataset_from_directory, load_img, img_to_array, array_to_img

import warnings
warnings.filterwarnings('ignore')

In [2]:
# First, I want to get a random image to understand what they look like and their properties 
rand_image_path = "chest_xray/train/NORMAL/" + random.choice(os.listdir("chest_xray/train/NORMAL"))
rand_image = load_img(rand_image_path)
rand_image.getpixel

<bound method Image.getpixel of <PIL.Image.Image image mode=RGB size=1620x1438 at 0x1A4CF44EF10>>

In [3]:
# Directory path
test_folder = 'chest_xray/test'
train_folder = 'chest_xray/train'

In [4]:
batch_size = 32
img_height = 256
img_width = 256

In [23]:
train_ds = image_dataset_from_directory(
  train_folder,
  label_mode = 'binary',
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 5232 files belonging to 2 classes.
Using 4186 files for training.


In [22]:
train_ds

<BatchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>

In [6]:
val_ds = image_dataset_from_directory(
  train_folder,
  label_mode = 'binary',
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 5232 files belonging to 2 classes.
Using 1046 files for validation.


In [7]:
train_ds

<BatchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>

In [8]:
test_ds = image_dataset_from_directory(
  test_folder,
  label_mode = 'binary',
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 624 files belonging to 2 classes.


In [9]:
class_names = train_ds.class_names
print(class_names)

['NORMAL', 'PNEUMONIA']


In [24]:
# Build a baseline fully connected model
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

num_classes = len(class_names)

model = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Flatten(),
  layers.Dense(20, activation='relu'),
  layers.Dense(7, activation='relu'),
  layers.Dense(num_classes, activation='softmax')
])

In [37]:
# code from: https://datascience.stackexchange.com/questions/45165/how-to-get-accuracy-f1-precision-and-recall-for-a-keras-model

from tensorflow.keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [38]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['acc', f1_m])

history = model.fit(train_ds,
                    epochs=50,
                    validation_data=val_ds)

Epoch 1/50
131/131 [==============================] - 8s 58ms/step - loss: 0.0861 - acc: 0.9732 - f1_m: 0.8446 - val_loss: 0.0898 - val_acc: 0.9704 - val_f1_m: 0.8702
Epoch 2/50
131/131 [==============================] - 8s 58ms/step - loss: 0.0762 - acc: 0.9730 - f1_m: 0.8444 - val_loss: 0.0979 - val_acc: 0.9637 - val_f1_m: 0.8698
Epoch 3/50
131/131 [==============================] - 8s 59ms/step - loss: 0.0874 - acc: 0.9713 - f1_m: 0.8442 - val_loss: 0.0897 - val_acc: 0.9723 - val_f1_m: 0.8704
Epoch 4/50
131/131 [==============================] - 8s 57ms/step - loss: 0.0812 - acc: 0.9761 - f1_m: 0.8439 - val_loss: 0.1089 - val_acc: 0.9598 - val_f1_m: 0.8711
Epoch 5/50
131/131 [==============================] - 8s 60ms/step - loss: 0.1008 - acc: 0.9637 - f1_m: 0.8442 - val_loss: 0.0966 - val_acc: 0.9646 - val_f1_m: 0.8716
Epoch 6/50
131/131 [==============================] - 8s 59ms/step - loss: 0.0874 - acc: 0.9680 - f1_m: 0.8440 - val_loss: 0.1233 - val_acc: 0.9570 - val_f1_m: 0.871

Epoch 50/50
131/131 [==============================] - 9s 64ms/step - loss: 0.0981 - acc: 0.9680 - f1_m: 0.8450 - val_loss: 0.1091 - val_acc: 0.9675 - val_f1_m: 0.8714


In [26]:
results_train = model.evaluate(train_ds)

131/131 [==============================] - 6s 41ms/step - loss: 0.0960 - accuracy: 0.9656


In [27]:
results_train

[0.09600621461868286, 0.9655995965003967]

In [28]:
results_val = model.evaluate(val_ds)

33/33 [==============================] - 1s 33ms/step - loss: 0.1184 - accuracy: 0.9589


In [30]:
results_val

[0.11843656003475189, 0.9588910341262817]